In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import dask.array
from zarr.storage import FSStore
import fsspec
from fsspec.implementations.local import LocalFileSystem

In [2]:
fs = fsspec.filesystem(
    protocol="az",
    account_name="sauchdev",
    anon=False,
)

remote_store = FSStore(
    url="az://data/versioned_nwp",
    fs=fs,
)

In [ ]:
fs.download("az://data/zarr_templates/versioned_nwp_sample.nc", "versioned_nwp_sample_001.nc")

sample = xr.open_dataset("versioned_nwp_sample_001.nc")

[None]

In [3]:
template = xr.Dataset(None, coords=sample.coords)

In [4]:
for name in template._coord_names:
    if np.issubdtype(template[name].dtype, np.datetime64):
        template[name].encoding["units"] = "hours since 2023-01-01 00:00:00"
    if np.issubdtype(template[name].dtype, np.timedelta64):
        template[name].encoding["units"] = "hours"

In [5]:
horizon_idx = pd.TimedeltaIndex([pd.Timedelta(hours=i) for i in range(61)])

In [6]:
template = template.reindex(horizon=horizon_idx)

In [7]:
model_run_time_utc_idx = pd.date_range(start="2023-01-01T00:00:00", end="2026-01-01T00:00:00", freq=pd.Timedelta(hours=3))

In [8]:
template = template.reindex(model_run_time_utc=model_run_time_utc_idx)

In [9]:
shape = tuple(v for v in template.coords.sizes.values())

In [10]:
dims = tuple(k for k in template.dims.keys())

/tmp/ipykernel_9943/3042545308.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  dims = tuple(k for k in template.dims.keys())


In [11]:
empty_array = dask.array.empty(shape=shape, chunks=[shape[0], 1, 1, shape[3], shape[4]])

In [12]:
for variable in sample.data_vars:
    template[variable] = (dims, empty_array)

In [15]:
template.to_zarr(
    store=remote_store,
    compute=False,
)

Delayed('_finalize_store-4bf31f75-c5e8-4d3d-a24c-5fdb46e8d57e')